In [1]:
import os
import cv2
import keras 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras import regularizers
import pickle
import keras 
from keras import backend as K
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import Activation, MaxPooling2D
from keras.layers.core import Dense, Flatten
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.convolutional import *
from sklearn.metrics import confusion_matrix
from keras.constraints import max_norm
from keras import optimizers
from sklearn.metrics import precision_score, recall_score, f1_score
from keras.callbacks import ReduceLROnPlateau

In [2]:
# To mount drive, if you are at colab
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


# Import Data

In [3]:
data_dir     = '/content/gdrive/MyDrive/dataset anemia/'
india_folder = os.path.join(data_dir, 'India')
italy_folder = os.path.join(data_dir, 'Italy')

In [4]:
def should_use_img(img_name, img_type):
    if img_name.split(".")[1] in ["jpg", "png"]:
        img_name = img_name.split(".")[0]
        
        if img_type == "":
            img_types   = ["forniceal", "forniceal_palpebral", "palpebral"]
            for _type in img_types:
                if img_name.endswith(_type):
                    return False
            return True
        elif img_type == "palpebral":
            img_name = img_name.split(".")[0]
            if img_name.endswith("forniceal_palpebral"):
                    return False
            return True if img_name.endswith(img_type) else False
            
        else:
            return True if img_name.endswith(img_type)  else False
    
    else:
        return False
        

img_to_use  = "forniceal_palpebral"
img_files   = {"India":[], "Italy":[]}

for folder in ['India', 'Italy']:
    img_folder = os.path.join(data_dir, folder)
    print("Looking into -> ", folder)
    for root, dirs, files in os.walk(img_folder):
        flag = False
        for file in files:
            if should_use_img(file, img_to_use):
                cls = root.split("\\")[-1]
                img_files[folder].append( ( os.path.join(root, file), cls) )
                flag = True
                break
        if flag is False:
            print(root, " not found -> ", files)


Looking into ->  India
/content/gdrive/MyDrive/dataset anemia/India  not found ->  ['.DS_Store', 'IndiaRd.xlsx']
Looking into ->  Italy
/content/gdrive/MyDrive/dataset anemia/Italy  not found ->  ['.DS_Store', 'Italyrd.xlsx']
/content/gdrive/MyDrive/dataset anemia/Italy/75  not found ->  ['T_44_20190611_083543.jpg', 'T_44_20190611_083543_palpebral.png', '.DS_Store']
/content/gdrive/MyDrive/dataset anemia/Italy/58  not found ->  ['T_27_20190608_100451_palpebral.png', 'T_27_20190608_100451.jpg', '.DS_Store']
/content/gdrive/MyDrive/dataset anemia/Italy/54  not found ->  ['T_23_20190608_090427.jpg', 'T_23_20190608_090427_palpebral.png', '.DS_Store']
/content/gdrive/MyDrive/dataset anemia/Italy/35  not found ->  ['T_4_20190606_095326.jpg', 'T_4_20190606_095326_palpebral.png', '.DS_Store']
/content/gdrive/MyDrive/dataset anemia/Italy/109  not found ->  ['T_78_20190614_074753.jpg', 'T_78_20190614_074753_palpebral.png', '.DS_Store']
/content/gdrive/MyDrive/dataset anemia/Italy/1  not found ->

In [5]:
for key, items in img_files.items():
    print(f"items for {key} -> {len(items)}")

items for India -> 95
items for Italy -> 117


In [6]:
img_files["India"][0]

('/content/gdrive/MyDrive/dataset anemia/India/95/20200318_130225_forniceal_palpebral.png',
 '/content/gdrive/MyDrive/dataset anemia/India/95')

In [7]:
data_dir = '/content/gdrive/MyDrive/dataset anemia/'
file1    = data_dir+'India/IndiaRd.xlsx'
file2    = data_dir+'Italy/Italyrd.xlsx'

In [8]:

labels = {}

d1 = pd.read_excel(file1)
d1["Anemia"] = d1["Note"]
d1 = d1[["Number", "Anemia" ]]
d1.replace("No anemia", "No Anemia", inplace=True)

d2 = pd.read_excel(file2)[["Number", "Anemia" ]]
d2.replace("No anemia", "No Anemia", inplace=True)

unique_values   = list(d1['Anemia'].unique())
enum            = {val:i for i, val in enumerate(unique_values)} 
print(enum)

d1['Anemia'].replace(enum, inplace=True)
d2['Anemia'].replace(enum, inplace=True)

labels["India"] = { row["Number"]:row["Anemia"] for index, row in d1.iterrows() }
labels["Italy"] = { row["Number"]:row["Anemia"] for index, row in d2.iterrows() }


{'No Anemia': 0, 'Anemia': 1}


# Read Images

In [9]:

X = []
y = []


for key, items in img_files.items():
    for item in items:
        img_path, cls = item
        cls           = cls.split("/")[-1]
        img           = cv2.imread(img_path)
        img           = cv2.resize(img, (64, 64))
        img           = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        kernel        = np.array([[-1,-1,-1], 
                                    [-1, 9,-1],
                                    [-1,-1,-1]])
        sharpened     = cv2.filter2D(img, -1, kernel)
        img           = cv2.GaussianBlur(sharpened, (5, 5), 0)
        img           = img.reshape(64, 64, 1)
        lbl           = labels[key][int(cls)]
        
        X.append(img)
        y.append(lbl)
        

In [112]:
print(f"len(X) -> {len(X)}")
print(f"len(y) -> {len(y)}")

len(X) -> 212
len(y) -> 212


In [113]:
y = [0 if i == 0 else 1 for i in y]

In [114]:
X = np.array(X)
X.shape

(212, 64, 64, 1)

In [115]:
set(y)

{0, 1}

In [116]:
number_classes = len(set(y))
number_classes

2

In [117]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, train_size=0.8, random_state=0)
#X_test, X_val, Y_test, Y_val     = train_test_split(X_test, Y_test, train_size=0.9, random_state=123)

# Augmentation

In [123]:
X_train.shape

(169, 64, 64, 1)

In [124]:
characters  = X_train
labels      = np.array(Y_train.copy()) 

In [125]:
datagen = ImageDataGenerator( width_shift_range=[-200,200],
    rotation_range=30,
    zoom_range=0.15,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")



In [126]:
iterator = datagen.flow(characters, labels)

In [127]:
set(labels)

{0, 1}

In [128]:
label_counts = [(lbl, list(labels).count(lbl) ) for lbl in set(labels)]
label_counts

[(0, 97), (1, 72)]

In [129]:
augment_lbls = [0, 1]
NUM = 1500
sample_count = {item:0 for item in augment_lbls}
x_aug = []
y_aug = []
while True:
    x_tmp, y_tmp = iterator.next()
    for x, y in zip(x_tmp, y_tmp):
    if y in augment_lbls:
        if sample_count[y] < NUM:
            x_aug.append(x)
            y_aug.append(y)
            sample_count[y] += 1
    if sum([sample_count[key] >= NUM for key in sample_count.keys()]) == len(sample_count.keys()):
    break

In [130]:
set(y_aug)

{0, 1}

In [131]:
label_counts = [(lbl, y_aug.count(lbl) ) for lbl in set(y_aug)]
label_counts

[(0, 1500), (1, 1500)]

In [132]:
np.array(x_aug).shape

(3000, 64, 64, 1)

In [133]:
type(characters)

numpy.ndarray

In [134]:
characters.shape

(169, 64, 64, 1)

In [135]:
characters = list(characters)
characters.extend(x_aug)
characters = np.array(characters)
characters.shape

(3169, 64, 64, 1)

In [136]:
labels = list(labels)
labels.extend(y_aug)
len(labels)

3169

In [137]:
label_counts = [(lbl, labels.count(lbl) ) for lbl in set(labels)]
label_counts

[(0, 1597), (1, 1572)]

# Data Spliting

In [138]:
characters.shape

(3169, 64, 64, 1)

In [139]:
X = characters
Y = labels
number_classes = len(set(labels))

In [140]:

X_train, X_val, Y_train, Y_val= train_test_split(X, Y, train_size=0.8, random_state=0)

In [141]:
from keras.utils import np_utils

Y_train = keras.utils.np_utils.to_categorical(Y_train, number_classes)
Y_test = keras.utils.np_utils.to_categorical(Y_test, number_classes)
Y_val = keras.utils.np_utils.to_categorical(Y_val, number_classes)

#Y = keras.utils.to_categorical(Y, number_classes)

In [142]:
print(len(X_train))
print(len(X_val))
print(len(X_test))

2535
634
43


# Create Model

In [143]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization

In [174]:

input_shape = X_test[0].shape
drop_rate= 0.
k3=(3, 3)
k5=(5, 5)
k7= (7, 7)
model = Sequential(layers=[
    #1st Convolutional layer
    Conv2D(filters=256, kernel_size=k5, padding='same', activation='relu', input_shape=input_shape),  
    BatchNormalization(),
    MaxPooling2D(pool_size=(3, 3), strides=None, padding='valid', data_format=None),
    Dropout(rate=drop_rate),
    #2nd Convolutional layer
    Conv2D(filters=128, kernel_size=k3, padding='same', activation='relu'), 
    BatchNormalization(),
    Dropout(rate=drop_rate),
    #3rd Convolutional layer
    Conv2D(filters=128, kernel_size=k3, padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(3, 3), strides=None, padding='valid', data_format=None),
    Dropout(rate=drop_rate),
    
    #Flat layer
    Flatten(), 
    
    #6th Layer
    Dense(4096, activation='relu'),
    BatchNormalization(),
    Dropout(drop_rate),
    
    Dense(number_classes, activation='softmax')
])

In [175]:

model.compile(optimizer=Adam(learning_rate=.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [176]:
!rm -r models
import os
os.makedirs('./models')

# Fit

In [179]:
i =0
epochs = 10
train_loss = []
val_loss = []
val_acc = []
train_acc = []
while i <= epochs:
    print(i)
    hist = model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=1, shuffle=True, batch_size=32) #callbacks=[learning_rate_reduction]) 
    model.save_weights('./models/model'+str(i)+'.hd5')
    i = i+1
  #break

0
80/80 [==============================] - 3s 38ms/step - loss: 0.5579 - accuracy: 0.6536 - val_loss: 0.7637 - val_accuracy: 0.5079
1
80/80 [==============================] - 3s 34ms/step - loss: 0.5739 - accuracy: 0.6418 - val_loss: 0.8628 - val_accuracy: 0.5142
2
80/80 [==============================] - 3s 34ms/step - loss: 0.5756 - accuracy: 0.6414 - val_loss: 0.8027 - val_accuracy: 0.4921
3
80/80 [==============================] - 3s 33ms/step - loss: 0.5636 - accuracy: 0.6355 - val_loss: 0.8300 - val_accuracy: 0.4984
4
80/80 [==============================] - 3s 32ms/step - loss: 0.5693 - accuracy: 0.6272 - val_loss: 0.7542 - val_accuracy: 0.5489
5
80/80 [==============================] - 3s 33ms/step - loss: 0.5596 - accuracy: 0.6343 - val_loss: 0.7647 - val_accuracy: 0.5442
6
80/80 [==============================] - 3s 34ms/step - loss: 0.5600 - accuracy: 0.6584 - val_loss: 0.7746 - val_accuracy: 0.5489
7
80/80 [==============================] - 3s 34ms/step - loss: 0.5415 - acc

In [180]:

i = 0
precission, recall, f1_, test_acc, test_loss = ([] for i in range(5))
while i <= epochs:
   
    if i%1 == 0:
        model.load_weights('./models/model' + str(i) + '.hd5')
        predict_x=model.predict(X_test) 
        yp       =np.argmax(predict_x,axis=1) 
        #yp = model.predict_classes(X_test)
        ya = [np.where(p==1)[0][0] for p in Y_test]

        score = model.evaluate(X_test, Y_test)
        pre = precision_score(ya, yp, average='weighted')
        rec = recall_score(ya, yp, average='macro')
        f1 = f1_score(ya, yp, average='micro') 
        test_acc.append(score[1])
        test_loss.append(score[0])
        precission.append(pre)
        recall.append(recall)
        f1_.append(f1)
        print( str(i)+': Testing Score: '+str(score[1]*100) + ', Loss: ' + str(score[0]))
    i = i + 1

2/2 [==============================] - 0s 13ms/step - loss: 0.9351 - accuracy: 0.7442
0: Testing Score: 74.41860437393188, Loss: 0.9351410865783691
2/2 [==============================] - 0s 12ms/step - loss: 0.7403 - accuracy: 0.7674
1: Testing Score: 76.74418687820435, Loss: 0.7402984499931335
2/2 [==============================] - 0s 16ms/step - loss: 0.5812 - accuracy: 0.7442
2: Testing Score: 74.41860437393188, Loss: 0.5811589956283569
2/2 [==============================] - 0s 12ms/step - loss: 0.9055 - accuracy: 0.6977
3: Testing Score: 69.76743936538696, Loss: 0.905520498752594
2/2 [==============================] - 0s 12ms/step - loss: 0.8312 - accuracy: 0.6977
4: Testing Score: 69.76743936538696, Loss: 0.831231415271759
2/2 [==============================] - 0s 12ms/step - loss: 1.0750 - accuracy: 0.6744
5: Testing Score: 67.44186282157898, Loss: 1.0750120878219604
2/2 [==============================] - 0s 11ms/step - loss: 0.7520 - accuracy: 0.7442
6: Testing Score: 74.4186043

In [166]:
print('Acc: ' + str(max(test_acc))+ ' at '+ str(np.argmax(test_acc)) )
print('loss: '+ str(min(test_loss))+ ' at '+ str(np.argmax(test_loss)) )

Acc: 0.8604651093482971 at 5
loss: 0.40779370069503784 at 1


In [167]:
model.load_weights('./models/model' + str(5) + '.hd5')

In [168]:
model.evaluate(X_test, Y_test)

2/2 [==============================] - 0s 13ms/step - loss: 0.4219 - accuracy: 0.8605


[0.42191389203071594, 0.8604651093482971]

# Save Model For Later Use

In [169]:
model.save('model_forniceal_palp_86_04_35.hd5')

INFO:tensorflow:Assets written to: model_forniceal_palp_86_04_35.hd5/assets


In [ ]:
print('Acc: ' + str(max(test_acc))+ ' at '+ str(np.argmax(test_acc)) )
print('loss: '+ str(min(test_loss))+ ' at '+ str(np.argmax(test_loss)) )

In [181]:
!zip -r model_forniceal_palp_86_04_35.zip model_forniceal_palp_86_04_35.hd5/

  adding: model_forniceal_palp_86_04_35.hd5/ (stored 0%)
  adding: model_forniceal_palp_86_04_35.hd5/keras_metadata.pb (deflated 93%)
  adding: model_forniceal_palp_86_04_35.hd5/variables/ (stored 0%)
  adding: model_forniceal_palp_86_04_35.hd5/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: model_forniceal_palp_86_04_35.hd5/variables/variables.index (deflated 72%)
  adding: model_forniceal_palp_86_04_35.hd5/assets/ (stored 0%)
  adding: model_forniceal_palp_86_04_35.hd5/saved_model.pb (deflated 89%)


In [182]:
!mv model_forniceal_palp_86_04_35.zip /content/gdrive/MyDrive/

In [ ]:
from tensorflow.keras.models import load_model

model1 = load_model('model_palpebral_93_18.hd5')

In [ ]:
model1.evaluate(X_test, Y_test)